In [1]:
import os
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

yf.pdr_override()

# Zipline imports
from zipline.utils.run_algo import load_extensions
from zipline.data import bundles
from zipline.utils.calendar_utils import get_calendar

from zipline.utils.calendar_utils import get_calendar
from zipline.api import(order_value,
                        symbol,
                        order_target_percent,
                        get_open_orders)
from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar
from zipline.api import symbol # type: ignore

# Ignore Warnings  
import warnings
warnings.filterwarnings('ignore', category=Warning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import generate_signal as gs

In [4]:
bundle = 'sp50'  # rename the bundle same as your ingested bundle with zipine
calendar_name = 'NYSE'
calendar = get_calendar(calendar_name)
bundle_data = bundles.load(bundle)
sids = bundle_data.asset_finder.sids
assets = bundle_data.asset_finder.retrieve_all(sids)
sp50tickers = [coin.symbol for coin in assets]

In [5]:
START_DATE = pd.Timestamp('2018-01-01')  # change it to your desired time range
END_DATE = pd.Timestamp('2018-08-30')
BASE_CAPITAL = 100_000

In [ ]:
def initialize(context):
    context.idx = 0
    context.tickers = sp50tickers[:1]
    context.window = 50
    context.buy_stocks = set()
    context.cash_pct = 0.25

# Define the handle_data function
def handle_data(context, data):
    context.idx += 1
    if context.idx < context.window:
        return

    # Execute trades based on the generated signals
    for ticker in context.tickers:
        prices = data.history(symbol(ticker), "price", bar_count=context.window, frequency="1d")
        # Module is being called to generate signals
        signal = gs.group1_ensemble_model_signals(prices)
        open_orders = get_open_orders(ticker)

        if ticker not in open_orders:
            if signal == 1 and ticker not in context.buy_stocks:
                if context.portfolio.cash > 0:
                    order_value(symbol(ticker), context.portfolio.cash * context.cash_pct)  # Buy Signal
                    context.buy_stocks.add(ticker)

            elif signal == -1 and ticker in context.buy_stocks:
                order_target_percent(symbol(ticker), 0)  # Sell Signal
                context.buy_stocks.remove(ticker)

# Run the algorithm
results = run_algorithm(
    start=START_DATE,
    end=END_DATE,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=BASE_CAPITAL,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='sp50',
)
gs.plots(results)